# Demo 2 — Hyperparameter Tuning with Optuna

Use Optuna to search for the best ARIMA `(p, d, q)` parameters with **Fourier seasonal features**.
Each trial is logged as a **nested MLflow run** — great for comparing results in the UI.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
import optuna
import mlflow
import mlflow.statsmodels
import warnings
warnings.filterwarnings("ignore")
from features import fourier_features, SEASONAL_PERIODS

In [2]:
mlflow.set_tracking_uri("http://localhost:5050")
mlflow.set_experiment("temperature-forecast-trial")

<Experiment: artifact_location='mlflow-artifacts:/5', creation_time=1772038374258, experiment_id='5', last_update_time=1772038374258, lifecycle_stage='active', name='temperature-forecast-trial', tags={}, workspace='default'>

## Load data & train/test split

In [3]:
df = pd.read_csv("../data/jena_daily_temp.csv", parse_dates=["Date Time"], index_col="Date Time")

train = df.iloc[:-90]
test = df.iloc[-90:]

ref_date = df.index[0]
exog_train = fourier_features(train.index, ref_date)
exog_test = fourier_features(test.index, ref_date)

print(f"Train: {len(train)} days | Test: {len(test)} days")
print(f"Fourier features: {list(SEASONAL_PERIODS.keys())} ({exog_train.shape[1]} columns)")

Train: 2831 days | Test: 90 days
Fourier features: ['year', 'month', 'bi_week', 'week'] (8 columns)


## Optuna objective with nested MLflow runs

In [4]:
def objective(trial):
    p = trial.suggest_int("p", 0, 8)
    d = trial.suggest_int("d", 0, 2)
    q = trial.suggest_int("q", 0, 8)
    order = (p, d, q)

    with mlflow.start_run(run_name=f"arima-{p}-{d}-{q}", nested=True):
        try:
            model = ARIMA(train["temperature"], order=order, exog=exog_train)
            results = model.fit()

            forecast = results.forecast(steps=len(test), exog=exog_test)
            rmse = np.sqrt(mean_squared_error(test["temperature"], forecast))

            mlflow.log_params({"p": p, "d": d, "q": q, "order": str(order)})
            mlflow.log_metrics({"rmse": rmse, "aic": results.aic})

            return rmse
        except Exception:
            return float("inf")

## Run the study (~50 trials)

In [5]:
with mlflow.start_run(run_name="optuna-tuning"):
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=10, show_progress_bar=True)

    # Log the best result to the parent run
    best = study.best_trial
    mlflow.log_params({
        "best_p": best.params["p"],
        "best_d": best.params["d"],
        "best_q": best.params["q"],
    })
    mlflow.log_metric("best_rmse", best.value)

print(f"Best order: ({best.params['p']}, {best.params['d']}, {best.params['q']})")
print(f"Best RMSE: {best.value:.2f}")

[I 2026-02-25 17:56:24,494] A new study created in memory with name: no-name-4ff8e91e-7628-498f-8da6-bcbbd03fad3f


  0%|          | 0/10 [00:00<?, ?it/s]

🏃 View run arima-0-0-5 at: http://localhost:5050/#/experiments/5/runs/c3ddc20222174283afd2accf57ab7d51
🧪 View experiment at: http://localhost:5050/#/experiments/5
[I 2026-02-25 17:56:27,248] Trial 0 finished with value: 3.654484809234206 and parameters: {'p': 0, 'd': 0, 'q': 5}. Best is trial 0 with value: 3.654484809234206.
🏃 View run arima-3-0-6 at: http://localhost:5050/#/experiments/5/runs/212dbd8fcd224e178059987eab068e14
🧪 View experiment at: http://localhost:5050/#/experiments/5
[I 2026-02-25 17:56:31,210] Trial 1 finished with value: 3.701753233392871 and parameters: {'p': 3, 'd': 0, 'q': 6}. Best is trial 0 with value: 3.654484809234206.
🏃 View run arima-2-0-8 at: http://localhost:5050/#/experiments/5/runs/042e815bb4a940a5b2cc75fa07af2ddd
🧪 View experiment at: http://localhost:5050/#/experiments/5
[I 2026-02-25 17:56:40,567] Trial 2 finished with value: 3.712489637343018 and parameters: {'p': 2, 'd': 0, 'q': 8}. Best is trial 0 with value: 3.654484809234206.
🏃 View run arima-8-

## Bonus: MLflowCallback

Optuna also provides a built-in `MLflowCallback` that logs trials automatically.

In [ ]:
from optuna.integration.mlflow import MLflowCallback

mlflow_callback = MLflowCallback(
    tracking_uri="http://localhost:5050",
    metric_name="rmse",
    create_experiment=False,
    mlflow_kwargs={"experiment_id": mlflow.get_experiment_by_name("temperature-forecast-simple").experiment_id},
)


def objective_simple(trial):
    p = trial.suggest_int("p", 0, 8)
    d = trial.suggest_int("d", 0, 2)
    q = trial.suggest_int("q", 0, 8)

    try:
        model = ARIMA(train["temperature"], order=(p, d, q), exog=exog_train)
        results = model.fit()
        forecast = results.forecast(steps=len(test), exog=exog_test)
        return np.sqrt(mean_squared_error(test["temperature"], forecast))
    except Exception:
        return float("inf")


study2 = optuna.create_study(direction="minimize")
study2.optimize(objective_simple, n_trials=10, callbacks=[mlflow_callback])

print(f"Best RMSE (callback approach): {study2.best_value:.2f}")

---
**Next:** Open the MLflow UI at http://localhost:5050, expand the parent run to see nested trials, and sort by RMSE.